In [3]:
import pdfplumber
import pandas as pd
import re
from collections import defaultdict
import os

def extract_recipes(text):
    pattern = r"(\b[A-Z0-9-]+\b)\s+RECIPE(.+?)(?=\b[A-Z0-9-]+\b\s+RECIPE|\Z)"
    recipe_blocks = re.findall(pattern, text, re.DOTALL)
    data = defaultdict(lambda: defaultdict(int))

    for product_title, recipe_block in recipe_blocks:
        lines = recipe_block.split("\n")
        product_codes = lines[0].split()

        for line in lines[1:]:
            parts = line.split()
            flower_color = " ".join(parts[:-len(product_codes)])

            # Only process the quantities if they can be converted to integers
            quantities = [part for part in parts[-len(product_codes):] if part.isdigit()]

            for product_code, quantity in zip(product_codes, quantities):
                data[(product_title, flower_color)][product_code] = int(quantity)

    return pd.DataFrame(data).T

def pdf_to_csv(pdf_path, csv_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for i in range(3, 16):  # Only process pages 4-16
            text += pdf.pages[i].extract_text()

    recipes_df = extract_recipes(text)
    recipes_df.to_csv(csv_path)

# Usage
pdf_to_csv('LookBook2022/3649 VDay22 LookBook - 2.pdf', 'CSVs/output.csv')


In [4]:
import pandas as pd

# Load the original output CSV
df = pd.read_csv("CSVs/output.csv", index_col=[0, 1])

# Reset the index
df_reset = df.reset_index()

# Rename the columns for clarity
df_reset.rename(columns={"level_0": "Product", "level_1": "Flower"}, inplace=True)

# Define the unwanted strings
unwanted_strings = ['BOUQUET', 'BASKET', 'ROSE BOUQUET', 'Market Price', '\$\d+', 'vase', 'Vase', '8" Red Glass']

# Combine the unwanted strings into a regular expression
regex = '|'.join(unwanted_strings)

# Remove the unwanted strings from the 'Flower' column
df_reset['Flower'] = df_reset['Flower'].str.replace(regex, '', regex=True).str.strip()

# Drop any row where 'Flower' contains no color names
color_names = ['hot pink', 'orange', 'pink', 'green', 'purple', 'yellow', 'white', 
               'peach', 'lavender', 'light pink', 'red', 'pale pink', 'blue', 'ivory']
df_reset = df_reset[df_reset['Flower'].str.contains('|'.join(color_names), case=False)]

# Drop the 'Product' column
df_reset.drop(columns='Product', inplace=True)

# Set 'Flower' as the index
df_cleaned = df_reset.set_index('Flower')

# Save the cleaned DataFrame to a new CSV file
df_cleaned.to_csv("CSVs/cleaned_output.csv")


In [5]:
# Load the cleaned output CSV
df = pd.read_csv("CSVS/cleaned_output.csv", index_col=[0, 1])

# Reset the index to make 'Flower' a column
df.reset_index(inplace=True)

# Drop any row where 'Flower' contains '8" Red Glass'
df = df[df['Flower'] != '8" Red Glass']

# Set 'Flower' as the index
df.set_index('Flower', inplace=True)

# Save the updated DataFrame to a new CSV file
df.to_csv("CSVs/updated_output.csv")


In [6]:
# Reshape the DataFrame back to its original form
df_reshaped = df_cleaned.stack().reset_index()

# Rename the columns for clarity
df_reshaped.columns = ['Flower', 'Product', 'Quantity']

# Extract base product name from 'Product'
df_reshaped['Product'] = df_reshaped['Product'].str.extract(r'(.*)[sdep]$', expand=False)

# Group the DataFrame by 'Flower' and 'Product' and sum the 'Quantity'
df_grouped = df_reshaped.groupby(['Flower', 'Product']).sum().reset_index()

# Pivot the grouped DataFrame to get the desired format
df_pivoted = df_grouped.pivot(index='Flower', columns='Product', values='Quantity')

# Fill NaN values with 0 and convert the quantities to integers
df_pivoted.fillna(0, inplace=True)
df_pivoted = df_pivoted.astype(int)

# Save the pivoted DataFrame to a new CSV file
df_pivoted.to_csv("CSVs/final_output.csv")


In [7]:
# Load the final output CSV
df = pd.read_csv("CSVs/final_output.csv", index_col=0)

# Define the color names
color_names = ['hot pink', 'orange', 'pink', 'green', 'purple', 'yellow', 'white', 
               'peach', 'lavender', 'light pink', 'red', 'pale pink', 'blue', 'ivory']

# Sort the color names in descending order of length
color_names_sorted = sorted(color_names, key=len, reverse=True)

# Create a new column for the color of the flowers
df['Color'] = None
for color in color_names_sorted:
    # Special handling for 'pink' to avoid matching 'hot pink'
    if color == 'pink':
        df.loc[df.index.str.contains(r'\b(?<!hot )' + color + r'\b', case=False, regex=True), 'Color'] = color
    else:
        df.loc[df.index.str.contains(r'\b' + color + r'\b', case=False, regex=True), 'Color'] = color

# Remove the color names from the 'Flower' column
for color in color_names_sorted:
    df.index = df.index.str.replace(r'\b' + color + r'\b', '', case=False, regex=True)

# Move the 'Color' column to the first position
df = df[ ['Color'] + [ col for col in df.columns if col != 'Color' ] ]

# Save the updated DataFrame to a new CSV file
df.to_csv("CSVs/final_output_no_color.csv")


In [ ]:
# Load the final output CSV
df = pd.read_csv("CSVs/final_output_no_color.csv", index_col=0)

# Create a grid-like DataFrame where the quantities are replaced with 1 for presence and 0 for absence
df_presence = df.copy()
df_presence.iloc[:, 1:] = (df_presence.iloc[:, 1:] > 0).astype(int)  # Skip the 'Color' column

# Write the quantities and presence DataFrames to separate sheets in the same Excel file
with pd.ExcelWriter("VDay22 flower grid.xlsx") as writer:  
    df.to_excel(writer, sheet_name='Quantities')
    df_presence.to_excel(writer, sheet_name='Presence')